In [2]:
# GOOGLE DRIVE PDF TO EXCEL GENERATOR
# Target Folder: 170DErJsrMpVN76SlWYvuyNNbJbWAtdQj

import pandas as pd
from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default

# --- CONFIGURATION ---
TARGET_FOLDER_ID = "1xnDfhWsbv0PGVX0SRIMuLZBttUkVzssg"
OUTPUT_FILENAME = "Google_Drive_PDF_List.xlsx"

# Step 1: Authentication
print("Authenticating...")
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

print(f"Target ID: {TARGET_FOLDER_ID}")
print("Scanning start ho rahi hai... (Sirf PDFs collect ho rahi hain)")

# Step 2: Scanning Logic (Wohi purana powerful logic)
def scan_folder_recursive(folder_id):
    found_items = []
    folders_to_process = [folder_id]
    processed_count = 0

    while folders_to_process:
        current_id = folders_to_process.pop(0)

        page_token = None
        while True:
            try:
                # Query: Folders aur PDFs dhundo
                query = f"'{current_id}' in parents and trashed = false and (mimeType = 'application/vnd.google-apps.folder' or mimeType = 'application/pdf')"

                results = service.files().list(
                    q=query,
                    fields="nextPageToken, files(id, name, mimeType)",
                    includeItemsFromAllDrives=True,
                    supportsAllDrives=True,
                    pageSize=1000,
                    pageToken=page_token
                ).execute()

                items = results.get('files', [])

                for item in items:
                    # Agar PDF hai to list mein daal lo
                    if item['mimeType'] == 'application/pdf':
                        found_items.append(item)

                    # Agar Folder hai to process list mein daalo taaki uske andar bhi check karein
                    elif item['mimeType'] == 'application/vnd.google-apps.folder':
                        folders_to_process.append(item['id'])

                page_token = results.get('nextPageToken')
                if not page_token:
                    break
            except Exception as e:
                print(f"Skipping folder due to error: {e}")
                break

        processed_count += 1
        print(f"Folders Scanned: {processed_count} | PDFs Found: {len(found_items)}", end='\r')

    return found_items

# --- EXECUTION ---
raw_items = scan_folder_recursive(TARGET_FOLDER_ID)

print(f"\nScanning Complete! Total PDFs found: {len(raw_items)}")
print("Excel file taiyar kar raha hun...")

# Step 3: Data ko Excel format mein lana
data = []

for item in raw_items:
    title = item['name']
    file_id = item['id']
    # Sirf View Link generate kar rahe hain
    link = f"https://drive.google.com/file/d/{file_id}/view"

    # Dictionary bana kar list mein add karo (Columns: Titles, Links)
    data.append({
        "Titles": title,
        "Links": link
    })

# Step 4: Save to Excel using Pandas
if data:
    df = pd.DataFrame(data)

    # File save location
    output_path = f"/content/drive/MyDrive/{OUTPUT_FILENAME}"

    # Excel file write karo (index=False taaki 0,1,2 numbering wala column na aaye)
    df.to_excel(output_path, index=False)

    print(f"\nSUCCESS! Excel file ban gayi hai.")
    print(f"Location: {output_path}")

    # Auto Download
    files.download(output_path)
else:
    print("\nKoi PDF nahi mili is folder mein.")

Authenticating...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Target ID: 1xnDfhWsbv0PGVX0SRIMuLZBttUkVzssg
Scanning start ho rahi hai... (Sirf PDFs collect ho rahi hain)
Folders Scanned: 3 | PDFs Found: 56
Scanning Complete! Total PDFs found: 56
Excel file taiyar kar raha hun...

SUCCESS! Excel file ban gayi hai.
Location: /content/drive/MyDrive/Google_Drive_PDF_List.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>